In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers import Adam


# Define the dataset path
data_dir = '/kaggle/input/guawa-dataset/Guava Dataset (final)'

# Load datasets with one-hot encoding of labels
train_dataset = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(227, 227),
    batch_size=32,
    label_mode='categorical'  # Ensures one-hot encoding
)

val_dataset = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(227, 227),
    batch_size=32,
    label_mode='categorical'  # Ensures one-hot encoding
)

# Define the base model
base_model = EfficientNetB0(input_shape=(227, 227, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model initially

# Add custom classification layers with Batch Normalization and L2 regularization
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.BatchNormalization(),  # Batch Normalization layer
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # Assuming 5 classes
])

# Compile the model using categorical crossentropy
model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint_cb = ModelCheckpoint('best_model.keras', save_best_only=True)
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Learning rate scheduler
def lr_schedule(epoch):
    return 1e-3 * 10 ** (epoch / 20)

lr_scheduler = LearningRateScheduler(lr_schedule)

# Train the model initially with a learning rate scheduler
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler]
)

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-30]:  # Freeze all layers except the last 30
    layer.trainable = False

# Recompile the model for fine-tuning with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with fine-tuning and callbacks
fine_tune_history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[checkpoint_cb, early_stopping_cb, lr_scheduler]
)

# Save the final model
#model.save('efficientnet_soybean_final_model.keras')

# Load the test dataset for final evaluation
test_dataset = image_dataset_from_directory(
    data_dir,  # Same directory, or separate if you have a distinct test set
    validation_split=0.2,  # If using a separate test set, remove this line
    subset="validation",  # Use this if your test set is split from the original dataset
    seed=123,
    image_size=(227, 227),
    batch_size=32,
    label_mode='categorical'  # Ensure one-hot encoding is consistent
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)

print(f'Test Accuracy: {test_accuracy:.4f}')

model.save('guawa.keras')

Found 9130 files belonging to 3 classes.
Using 7304 files for training.
Found 9130 files belonging to 3 classes.
Using 1826 files for validation.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20


I0000 00:00:1730391062.522500     103 service.cc:145] XLA service 0x7f5d5c0023b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730391062.522605     103 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  5/229 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - accuracy: 0.5694 - loss: 1.3823   

I0000 00:00:1730391087.658024     103 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


229/229 ━━━━━━━━━━━━━━━━━━━━ 83s 194ms/step - accuracy: 0.8845 - loss: 0.6986 - val_accuracy: 0.9556 - val_loss: 0.4110 - learning_rate: 0.0010
Epoch 2/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.9431 - loss: 0.4098 - val_accuracy: 0.9611 - val_loss: 0.2936 - learning_rate: 0.0011
Epoch 3/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.9539 - loss: 0.2977 - val_accuracy: 0.9491 - val_loss: 0.2585 - learning_rate: 0.0013
Epoch 4/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.9498 - loss: 0.2621 - val_accuracy: 0.9682 - val_loss: 0.2105 - learning_rate: 0.0014
Epoch 5/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.9466 - loss: 0.2591 - val_accuracy: 0.9540 - val_loss: 0.2266 - learning_rate: 0.0016
Epoch 6/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.9524 - loss: 0.2568 - val_accuracy: 0.9660 - val_loss: 0.2076 - learning_rate: 0.0018
Epoch 7/20
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.9504 - loss: 0.2504 - v

In [2]:
import numpy as np
from sklearn.metrics import classification_report

# Get the true labels and predicted labels from the validation set
true_labels = []
predicted_labels = []

for images, labels in val_dataset:
    # Predict the labels for each batch of images
    predictions = model.predict(images)
    predicted_labels.extend(np.argmax(predictions, axis=1))
    true_labels.extend(np.argmax(labels.numpy(), axis=1))

# Generate the classification report
class_names = val_dataset.class_names  # Get the class names from the dataset
report = classification_report(true_labels, predicted_labels, target_names=class_names)

# Print the classification report
print(report)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━